In [1]:
!python3 -m pip install faiss-cpu

In [ ]:
# !pip uninstall clip
!pip install git+https://github.com/openai/CLIP.git

In [8]:
import clip

In [8]:
import roboflow

roboflow.login()

roboflow.download_dataset(dataset_url="https://universe.roboflow.com/team-roboflow/coco-128/dataset/2", model_format="coco", location="data/coco-128")

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


In [8]:
!pip3 install roboflow

141766.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [4]:
def create_faiss_index(embeddings):
    # Determine the dimensionality of the embeddings
    d = 512
    
    # Initialize a FAISS index
    index = faiss.IndexFlatL2(d)

    # Add embeddings to the index
    index.add(embeddings)
    
    return index

def load_faiss_index(index_path, image_paths_path):
    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load image paths
    with open(image_paths_path, "r") as f:
        image_paths = json.load(f)
    
    return index, image_paths

In [1]:
import glob
import json
from PIL import Image
from pathlib import Path

import clip
import faiss
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset


class ImageDataset(Dataset):
    def __init__(self, image_paths, preprocess):
        self.image_paths = image_paths
        self.preprocess = preprocess

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        image = self.preprocess(image)
        return image


def get_data_paths(dir: str | list[str], data_formats: list, prefix: str = '') -> list[str]:
    """
    Get list of files in a folder that have a file extension in the data_formats.

    Args:
      dir (str | list[str]): Dir or list of dirs containing data.
      data_formats (list): List of file extensions. Ex: ['jpg', 'png']
      prefix (str): Prefix for logging messages.

    Returns:
      A list of strings.
    """
    try:
        f = []  # data files
        for d in dir if isinstance(dir, list) else [dir]:
            p = Path(d)
            if p.is_dir():
                f += glob.glob(str(p / '**' / '*.*'), recursive=True)
            else:
                raise FileNotFoundError(f'{prefix}{p} does not exist')
        data_files = sorted(x for x in f if x.split('.')[-1].lower() in data_formats)
        return data_files
    except Exception as e:
        raise Exception(f'{prefix}Error loading data from {dir}: {e}') from e


def get_image_embeddings(data_dir, model_name="ViT-B/32", batch_size=32, device="cpu"):
    # Load the CLIP model
    model, preprocess = clip.load(model_name, device=device)
    
    # Create a dataset and dataloader
    image_paths = get_data_paths(data_dir, data_formats=["jpg", "jpeg", "png"])
    print(len(image_paths))
    dataset = ImageDataset(image_paths, preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    # List to store image embeddings
    image_embeddings = []

    # Process images in batches
    with torch.no_grad():
        for images in dataloader:
            images = images.to(device)
            embeddings = model.encode_image(images)
            embeddings /= embeddings.norm(dim=-1, keepdim=True)
            image_embeddings.append(embeddings.cpu().numpy())

    # Convert list to numpy array
    image_embeddings = np.vstack(image_embeddings)
    
    return image_embeddings, image_paths


def create_faiss_index(embeddings):
    # Determine the dimensionality of the embeddings
    d = 512
    
    # Initialize a FAISS index
    index = faiss.IndexFlatL2(d)
    
    # Add embeddings to the index
    index.add(embeddings)
    
    return index


embeddings, image_paths = get_image_embeddings("data/coco-128/train", device="cpu")
# index = create_faiss_index(embeddings)
# faiss.write_index(index, "data/index.faiss")
# with open("data/image_paths.json", "w") as f:
#     json.dump(image_paths, f, indent=4)
    
filepath_mapping = {idx: path for idx, path in enumerate(image_paths)}
with open("data/filepath.json", "w") as f:
    json.dump(filepath_mapping, f, indent=4)

128


In [14]:
def get_similar_images(images, index, image_paths, top_k=5, model_name="ViT-B/32", device="cpu"):
    # Load the CLIP model
    clip_model, clip_preprocess = clip.load(model_name, device=device)
    
    # Preprocess and get embeddings for input images
    processed_images = torch.stack([clip_preprocess(img) for img in images]).to(device)
    
    with torch.no_grad():
        embeddings = clip_model.encode_image(processed_images)
        embeddings /= embeddings.norm(dim=-1, keepdim=True)
        embeddings = embeddings.cpu().numpy()
    
    # Search for similar images
    D, I = index.search(embeddings, top_k)
    
    # Get the paths of the similar images
    similar_images = [[image_paths[i] for i in indices] for indices in I]
    print(similar_images)
    similar_images = [[Image.open(img_path) for img_path in paths] for paths in similar_images]
    
    return similar_images

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
index, image_paths = load_faiss_index("data/index.faiss", "data/image_paths.json")


query_images = ["data/coco-128/test/000000000034_jpg.rf.b518abdaed199dcb88854cf20fce8078.jpg", "data/coco-128/test/000000000283_jpg.rf.27927692baf616a7456bb3e24c21bfd7.jpg"]  # replace with your image paths
query_images = [Image.open(img_path) for img_path in query_images]
similar_images = get_similar_images(query_images , index, image_paths)

: 

In [3]:
import faiss
import json
import numpy as np
from PIL import Image

INDEX_PATH = "data/index.faiss"
IMAGE_PATHS_PATH = "data/image_paths.json"
NUM_SAMPLES = 2  # Number of samples to retrieve

def load_faiss_index(index_path, image_paths_path):
    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load image paths
    with open(image_paths_path, "r") as f:
        image_paths = json.load(f)
    
    return index, image_paths

def get_samples_from_index(index, image_paths, num_samples=NUM_SAMPLES):
    # Get the total number of entries in the index
    num_entries = index.ntotal
    
    # Select random indices to retrieve
    sample_indices = np.random.choice(num_entries, num_samples, replace=False)
    
    # Retrieve the embeddings and corresponding image paths
    sample_embeddings = []
    sample_image_paths = []
    
    for idx in sample_indices:
        sample_embeddings.append(index.reconstruct(int(idx)))
        sample_image_paths.append(image_paths[int(idx)])
    
    return sample_embeddings, sample_image_paths

def display_sample_images(sample_image_paths):
    for image_path in sample_image_paths:
        img = Image.open(image_path)
        img.show()

if __name__ == "__main__":
    # Load the FAISS index and image paths
    index, image_paths = load_faiss_index(INDEX_PATH, IMAGE_PATHS_PATH)
    
    # Get samples from the index
    sample_embeddings, sample_image_paths = get_samples_from_index(index, image_paths)
    
    # Print sample embeddings and their corresponding image paths
    for i, (embedding, path) in enumerate(zip(sample_embeddings, sample_image_paths)):
        print(f"Sample {i+1}:")
        print(f"Image Path: {path}")
        print(f"Embedding: {embedding}")
        print()

    # Optionally display the images
    display_sample_images(sample_image_paths)


Sample 1:
Image Path: data/coco-128/train/000000000201_jpg.rf.602495fdc61e3930f33684db99b1a869.jpg
Embedding: [-3.85854468e-02  3.29561457e-02  6.30148640e-03  2.11395156e-02
 -6.10916913e-02 -1.61578264e-02  3.40977758e-02  2.32377145e-02
  2.64391191e-02  5.91386110e-03  1.17889477e-03 -5.29484265e-02
  6.96308762e-02  6.99306466e-03  3.35405543e-02 -1.50703185e-03
 -4.21562493e-02  1.75634809e-02  6.61315536e-03 -5.56848384e-02
 -7.38576725e-02 -1.75457355e-03 -2.89652012e-02 -8.44154954e-02
 -3.45061021e-03  2.74566356e-02  1.92912761e-02  1.19840279e-02
  1.00874901e-02  6.96676271e-03 -2.43539605e-02 -2.04196777e-02
  3.61893768e-03 -2.17918474e-02 -7.32437707e-03 -5.00712404e-03
 -1.63388811e-03  2.04930082e-02 -3.62345227e-03  1.53049365e-01
  4.05504506e-05  1.12711443e-02 -1.97407082e-02 -2.49413531e-02
  1.20235095e-02 -8.90282765e-02 -1.46743041e-02  3.47275138e-02
 -3.19401873e-03  1.43277156e-03  1.04446113e-02  4.15458195e-02
  2.18413230e-02  2.49150000e-03  1.94522869e

In [3]:
import faiss
import clip
import json
import numpy as np
from PIL import Image

def get_image_embeddings(data_dir, model_name="ViT-B/32", batch_size=32, device="cpu"):
    # Load the CLIP model
    model, preprocess = clip.load(model_name, device=device)
    
    # Create a dataset and dataloader
    image_paths = get_data_paths(data_dir, data_formats=["jpg", "jpeg", "png"])
    print(len(image_paths))
    dataset = ImageDataset(image_paths, preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    # List to store image embeddings
    image_embeddings = []

    # Process images in batches
    with torch.no_grad():
        for images in dataloader:
            images = images.to(device)
            embeddings = model.encode_image(images)
            embeddings /= embeddings.norm(dim=-1, keepdim=True)
            image_embeddings.append(embeddings.cpu().numpy())

    # Convert list to numpy array
    image_embeddings = np.vstack(image_embeddings)
    
    return image_embeddings, image_paths

embeddings, image_paths = get_image_embeddings("data/coco-128/train", device="cpu")

# Create and save filepath mapping
filepath_mapping = {idx: path for idx, path in enumerate(image_paths)}
with open("data/filepath.json", "w") as f:
    json.dump(filepath_mapping, f, indent=4)

In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 844.0 kB/s eta 0:00:00 0:00:01
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 2.3 MB/s eta 0:00:0000:0100:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 2.7 MB/s eta 0:00:0000:010:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.7 MB/s eta 0:00:0000:0100:01


In [5]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

#Encode an image:
img_emb = model.encode(Image.open('/Users/haphuongthao/Downloads/1a50811a14ee1a67b1fd7d7648fd24dd.png'))

#Encode text descriptions
text_emb = model.encode(['Two dogs in the snow', 'A cat on a table', 'A picture of London at night'])

#Compute cosine similarities 
cos_scores = util.cos_sim(img_emb, text_emb)
print(cos_scores)


tensor([[0.3291, 0.1615, 0.1654]])


In [7]:
img_emb

array([ 3.08756739e-01,  2.10063294e-01,  2.69430280e-02,  1.65039718e-01,
        2.40337297e-01, -7.69843042e-01,  4.03935671e-01,  5.71388721e-01,
        3.39634240e-01,  2.21902728e-01,  1.48812220e-01, -5.67920089e-01,
        4.25063282e-01,  3.33459452e-02,  6.73464835e-01, -3.13531071e-01,
       -4.66620177e-01,  3.38239968e-02, -2.45608956e-01,  2.21370772e-01,
       -8.19203973e-01,  6.13314658e-03, -3.72245908e-04, -2.87813932e-01,
       -6.54090643e-02,  3.38623881e-01,  1.97611481e-01,  1.32096291e-01,
       -2.32240006e-01, -4.78348769e-02, -2.36601949e-01, -2.94281483e-01,
        2.82699645e-01,  5.63129187e-02, -7.15818644e-01,  1.76680297e-01,
       -9.05007198e-02,  9.56150517e-02,  7.43588507e-02,  1.85330644e-01,
       -5.81507564e-01,  4.78417158e-01,  3.88193160e-01, -3.79562110e-01,
        1.65250197e-01, -1.05219766e-01,  1.68450251e-01, -1.09135881e-01,
        1.73855707e-01,  1.15752339e-01,  4.62054461e-02,  1.34683609e-01,
       -2.12076545e-01,  

In [9]:
img_emb.size

512

In [10]:
img_emb

array([ 3.08756739e-01,  2.10063294e-01,  2.69430280e-02,  1.65039718e-01,
        2.40337297e-01, -7.69843042e-01,  4.03935671e-01,  5.71388721e-01,
        3.39634240e-01,  2.21902728e-01,  1.48812220e-01, -5.67920089e-01,
        4.25063282e-01,  3.33459452e-02,  6.73464835e-01, -3.13531071e-01,
       -4.66620177e-01,  3.38239968e-02, -2.45608956e-01,  2.21370772e-01,
       -8.19203973e-01,  6.13314658e-03, -3.72245908e-04, -2.87813932e-01,
       -6.54090643e-02,  3.38623881e-01,  1.97611481e-01,  1.32096291e-01,
       -2.32240006e-01, -4.78348769e-02, -2.36601949e-01, -2.94281483e-01,
        2.82699645e-01,  5.63129187e-02, -7.15818644e-01,  1.76680297e-01,
       -9.05007198e-02,  9.56150517e-02,  7.43588507e-02,  1.85330644e-01,
       -5.81507564e-01,  4.78417158e-01,  3.88193160e-01, -3.79562110e-01,
        1.65250197e-01, -1.05219766e-01,  1.68450251e-01, -1.09135881e-01,
        1.73855707e-01,  1.15752339e-01,  4.62054461e-02,  1.34683609e-01,
       -2.12076545e-01,  